# Synthetic Raspberry Generation
Using qim3d's volume generation to create a synthetic raspberry structure

This notebook demonstrates a pipeline for how to create realistic 3D synthetic raspberries by simulating the natural growth pattern of drupelets (the small spherical units that make up a raspberry). 

The pipeline follows these steps:
1. **Understanding volume generation parameters** - Exploration of how different parameters affect the appearance of individual drupelets
2. **Positioning Algorithm** - Generate 3D positions where the generated volumes should be placed
3. **Boundary Aware Placement** - Place individual druplets while handling overlapping volumes

In [ ]:
import qim3d
import numpy as np
from tqdm import tqdm

## 1. Understanding Qim3d's Volume Generation
Before creating the raspberry structure, we explore how qim3d's volume generation works. The interactive `ParameterVisualizer()` allows us to understand how different settings affect the appearance of a generated volume.

In [ ]:
viz = qim3d.generate.ParameterVisualizer(base_shape=(128,128,128), seed=0, grid_visible=True)

## 2. Generating Synthetic raspberry positions
This section implements the core algorithm for generating realistic raspberry drupelet positions. The `generate_raspberry_positions()` function creates a natural 3D arrangement that mimics how drupelets grow on real raspberries.

1. The algorithm distributes points on a sphere and then applies several transformations to make it look more realistic. 

2. All points above a specified threshold are excluded to create the hollow opening at the top of the raspberry. 

3. The area just bellow the opening is handled differently to produce a slight curve to make it look more realistic.

4. Each druplet gets a slightly different radius based on it's position and random jitter.

5. Small random offsets are also added to the position of each druplet to make the final pattern less uniform.

In [ ]:
def generate_raspberry_positions(core_radius, bead_radius, num_beads,
                                bead_radius_jitter=0, position_jitter=0, seed=None,
                                offset=(0, 0, 0), top_opening_threshold=0.8, rim_thickness=0.1):
    rng = np.random.default_rng(seed)
    positions, bead_radii = [], []
    offset = np.array(offset, dtype=int)
    
    def process_bead(x, y, z, i):
        """Helper function to process a single bead"""
        # Rim area calculations
        rim_start = top_opening_threshold - rim_thickness
        top_factor = bead_scale = 1.0
        
        if y > rim_start:
            rim_progress = (y - rim_start) / rim_thickness
            top_factor = 1.0 - 0.15 * rim_progress
            bead_scale = 0.9 - 0.1 * rim_progress
        elif y > 0.6:
            transition_factor = (y - 0.6) / (rim_start - 0.6)
            top_factor = 1.0 - 0.05 * transition_factor
            bead_scale = 1.0 - 0.1 * transition_factor
        
        # Calculate position
        this_bead_radius = max(1, int(bead_radius * bead_scale) + 
                              rng.integers(-bead_radius_jitter, bead_radius_jitter + 1))
        bead_radii.append(this_bead_radius)
        
        direction = np.array([x, y, z]) / np.linalg.norm([x, y, z])
        base_pos = (core_radius + this_bead_radius) * direction * top_factor
        
        # Add perpendicular jitter
        rand_vec = rng.normal(size=3)
        rand_vec -= rand_vec.dot(direction) * direction
        rand_vec /= np.linalg.norm(rand_vec) + 1e-8
        offset_vec = rand_vec * rng.integers(-position_jitter, position_jitter + 1)
        
        pos = np.round(base_pos + offset_vec).astype(int) + offset
        positions.append(tuple(pos))
    
    # Generate beads with extra candidates to account for skipped ones
    extra_beads = int(num_beads * (1 - top_opening_threshold) * 0.75)
    total_candidates = num_beads + extra_beads
    offset_val = 2.0 / total_candidates
    increment = np.pi * (3.0 - np.sqrt(5.0))
    
    # Main generation loop
    for i in range(total_candidates):
        if len(positions) >= num_beads:
            break
        y = ((i * offset_val) - 1) + (offset_val / 2)
        if y <= top_opening_threshold:  # Only process valid beads
            r = np.sqrt(1 - y * y)
            phi = i * increment
            process_bead(np.cos(phi) * r, y, np.sin(phi) * r, i)
    
    # Fill any remaining spots with random beads in valid area
    while len(positions) < num_beads:
        y = rng.uniform(-1, top_opening_threshold)
        r = np.sqrt(1 - y * y)
        phi = rng.uniform(0, 2 * np.pi)
        process_bead(np.cos(phi) * r, y, np.sin(phi) * r, len(positions))
    
    print(f"Generated exactly {len(positions)} beads.")
    return positions, bead_radii

The result of this is a list of 3D coordinates for where each druplet should be placed, along with the appropriate radius for each one

In [ ]:
positions, bead_radii = generate_raspberry_positions(
    core_radius=10,
    bead_radius=2,
    num_beads=100,
    bead_radius_jitter=1,
    position_jitter=1,
    seed=42,
    offset=(64, 64, 64),
    top_opening_threshold=0.85,
    rim_thickness=0.1
)

import plotly.graph_objects as go

# Extract x, y, z coordinates from positions
x_coords = [pos[0] for pos in positions]
y_coords = [pos[1] for pos in positions]
z_coords = [pos[2] for pos in positions]

# Create 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    mode='markers',
    marker=dict(
        size=5,
        color=list(range(len(positions))),
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Order in Array")
    ),
    text=[f'Order: {i}, Radius: {r}' for i, r in enumerate(bead_radii)],
    hovertemplate='Position: (%{x}, %{y}, %{z})<br>%{text}<extra></extra>'
)])

fig.update_layout(
    title='Raspberry Drupelet Positions',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube'
    ),
    width=800,
    height=600
)

fig.show()

## 3. Drupelet Placement
Once we have the positions, we need to place the individual drupelets in a way that creates natural boundaries when they touch. When placing a new drupelet, we check firts if it will overlap with existing drupelets. In areas where there is an overlap, the algorithm decides which drupelet should "win" each voxel. The drupelet with the closest center to a point takes precedence. 

In [ ]:

def overlapping_placement(collection, blob, position, labels_array=None, label_id=0):
    z, y, x = position
    # Calculate start and end positions
    start = np.array([z, y, x]) - np.array(blob.shape) // 2
    end = start + np.array(blob.shape)
    
    # Check if placement is within bounds
    within_bounds = (np.all(start >= 0) and
                    np.all(end <= np.array(collection.shape)))
    
    if within_bounds:
        # Get the current slice
        collection_slice = collection[start[0]:end[0], start[1]:end[1], start[2]:end[2]]
        
        # Create distance map from current blob center
        blob_center = np.array(blob.shape) // 2
        blob_coords = np.mgrid[0:blob.shape[0], 0:blob.shape[1], 0:blob.shape[2]]
        blob_distances = np.sqrt(np.sum((blob_coords - blob_center[:, None, None, None])**2, axis=0))
        
        # For overlapping regions, use the blob that's closer to its own center
        # This creates natural boundaries halfway between blob centers
        overlap_mask = (collection_slice > 0) & (blob > 0)
        
        if labels_array is not None:
            # Get existing label distances to determine which blob should "win"
            label_slice = labels_array[start[0]:end[0], start[1]:end[1], start[2]:end[2]]
            
            # For simplicity, use intensity as a proxy for distance from center
            # Higher intensity = closer to center = wins the boundary
            new_blob = np.where(overlap_mask & (blob >= collection_slice), blob, 0)
            keep_existing = overlap_mask & (blob < collection_slice)
        else:
            # Without labels, use a simpler approach - split the difference
            new_blob = np.where(overlap_mask, blob * 0.5 + collection_slice * 0.5, blob)
            keep_existing = False
        
        # Place blob where there's no conflict, and resolved conflicts
        no_conflict = ~overlap_mask
        final_blob = np.where(no_conflict, blob, new_blob)
        
        # Update collection
        collection[start[0]:end[0], start[1]:end[1], start[2]:end[2]] = np.maximum(
            collection_slice, final_blob
        )
        
        # Update labels if provided
        if labels_array is not None and label_id > 0:
            blob_mask = final_blob > 0
            new_labels = np.where((label_slice == 0) & blob_mask, label_id, label_slice)
            # Update contested areas based on which blob "won"
            contested_areas = overlap_mask & (blob >= collection_slice)
            new_labels = np.where(contested_areas, label_id, new_labels)
            labels_array[start[0]:end[0], start[1]:end[1], start[2]:end[2]] = new_labels
        
        return collection, True
    
    return collection, False

## 4. Raspberry Assembly
Each drupelet is generated as a small 3D volume using qim3d's volume generation with the parameters we found in step 1. The size of each drupelet varies based on the radius calculated during the positioning. To make the raspberry look more natural, slight variations are applied to the generation parameters.

In [ ]:


def create_raspberry(collection_shape=(200, 200, 200),
                             core_radius=20,
                             bead_radius=4,
                             num_beads=32,
                             center=None,
                             seed=42,
                             **volume_params):

    if center is None:
        center = tuple(s // 2 for s in collection_shape)

    # Generate raspberry positions using your function
    positions, bead_radii = generate_raspberry_positions(
        core_radius=core_radius,
        bead_radius=bead_radius, 
        num_beads=num_beads,
        bead_radius_jitter=1,
        position_jitter=2,
        seed=seed,
        offset=center
    )
    
    # Initialize volumes
    raspberry_volume = np.zeros(collection_shape, dtype=np.uint8)
    labels = np.zeros(collection_shape, dtype=np.uint8)
    
    # Set default volume generation parameters
    default_params = {
        'noise_scale': 0.0,           # Smooth drupelets
        'gamma': 0.6,                 # Sharp edges
        'threshold': 0.75,            # Dense berries
        'max_value': 240,             # Bright berries
    }
    default_params.update(volume_params)
    
    successful_positions = []
    rng = np.random.default_rng(seed)
    
    print(f"Creating raspberry with {len(positions)} drupelets...")
    
    # Place each drupelet with overlap allowed
    for i, pos in enumerate(tqdm(positions, desc="Placing drupelets")):
        # Use the bead radius for this specific drupelet
        if i < len(bead_radii):
            drupelet_size = max(6, bead_radii[i] * 2)  # Convert radius to diameter
        else:
            drupelet_size = 8
            
        # Generate individual drupelet
        drupelet_shape = (drupelet_size, drupelet_size, drupelet_size)
        
        # Add some variation to each drupelet
        varied_params = default_params.copy()
        varied_params.update({
            'gamma': default_params['gamma'] + rng.uniform(-0.1, 0.1),
            'threshold': default_params['threshold'] + rng.uniform(-0.05, 0.05),
            'max_value': int(default_params['max_value'] + rng.integers(-20, 21))
        })
        
        drupelet = qim3d.generate.volume(
            base_shape=drupelet_shape,
            **varied_params
        )
        
        # Place drupelet with overlap allowed
        raspberry_volume, placed = overlapping_placement(
            raspberry_volume, drupelet, pos, labels, i + 1
        )
        
        if placed:
            successful_positions.append(pos)
    
    print(f"✅ Successfully placed {len(successful_positions)}/{len(positions)} drupelets")
    
    return raspberry_volume, labels, successful_positions



In [ ]:
raspberry, labels, positions = create_raspberry(
    collection_shape=(200, 200, 200),
    core_radius=20,
    bead_radius=15,
    num_beads=60,
    center=(100, 100, 100),
    seed=42,
    
    # Your exact parameters from the notebook
    noise_scale=0.02,           
    decay_rate=9.90,
    gamma=0.6,
    threshold=0.5,
    max_value=240,
)

The completed raspberry can now be visualised in multiple ways using the `qim3d.viz` functions. The `.viz.volumetric()` function show the complete raspberry as a solid 3D object, revealing the overal shape and arrangement.

In [ ]:
qim3d.viz.volumetric(raspberry)

The `.viz.slicer_orthogonal()` function allows examination of the internal structure by viewing slices through different orthogonal planes.

In [ ]:
qim3d.viz.slicer_orthogonal(raspberry, color_map='Reds')

We can even use `viz.colormaps.segmentation()` which uses the drupelet labels to create a colour map, making it easier to visualise the drupelet boundaries more clearly.

In [ ]:
num_drupelets = len(np.unique(labels))
cmap = qim3d.viz.colormaps.segmentation(num_labels=num_drupelets)
qim3d.viz.slicer_orthogonal(labels, color_map=cmap, value_max=num_drupelets)